In [1]:
import os
%pwd

'c:\\Users\\dengz\\Desktop\\Projects\\cnn_classifier\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\dengz\\Desktop\\Projects\\cnn_classifier'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list  # 在 PyTorch ResNet 中未直接使用，但保留以兼容配置
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [4]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import torch
import torch.nn as nn
import torchvision.models as models

In [7]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        create_directories([self.config.root_dir])

    def get_base_model(self):
        # 根据配置加载 ResNet-18，预训练权重（ImageNet）或随机初始化
        weights = None
        if self.config.params_weights.lower() == "imagenet":
            weights = models.ResNet18_Weights.IMAGENET1K_V1
        model = models.resnet18(weights=weights)

        # 若不包含顶层，全连接层替换为空
        if not self.config.params_include_top:
            model.fc = nn.Identity()

        self.base_model = model
        # 保存基础模型的 state_dict
        self.save_model(self.config.base_model_path, model)

    def update_base_model(self):
        model = self.base_model
        # 冻结所有层参数
        for param in model.parameters():
            param.requires_grad = False

        # 构建新的全连接分类层
        # ResNet-18 默认 fc.in_features 为 512
        in_features = model.fc.in_features if hasattr(model.fc, "in_features") else 512
        model.fc = nn.Linear(in_features, self.config.params_classes)

        # 仅分类层参数可训练，并初始化优化器
        optimizer = torch.optim.SGD(model.fc.parameters(), lr=self.config.params_learning_rate)

        self.full_model = model
        self.optimizer = optimizer

        # 保存更新后的模型 state_dict
        self.save_model(self.config.updated_base_model_path, model)

    @staticmethod
    def save_model(path: Path, model: torch.nn.Module):
        torch.save(model.state_dict(), path)




In [8]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2025-04-25 20:50:24,302: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-25 20:50:24,306: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-25 20:50:24,308: INFO: common: created directory at: artifacts]
[2025-04-25 20:50:24,309: INFO: common: created directory at: artifacts/prepare_base_model]
[2025-04-25 20:50:24,310: INFO: common: created directory at: artifacts\prepare_base_model]
